<a href="https://colab.research.google.com/github/Sh-Hi-Go/DTDL_findings/blob/main/Vector_Search_withFAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Failed attempts at using faiss for vector similarity search

In [1]:
!pip install langchain
!pip install sentence-transformers
!pip install faiss-gpu
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
df = pd.read_csv("/content/intern_data_final - intern_data.csv")
print(len(df))
df.head()

45


,question,combined_response
0,How can I implement user authentication in a M...,Consider using industry-standard libraries lik...
1,How to best structure a large React applicatio...,"Utilize modular design principles, break down ..."
2,What are some effective approaches to optimize...,Explore state management libraries like Redux ...
3,How can I create accessible and inclusive user...,"Follow WCAG guidelines, use semantic HTML elem..."
4,What are the key considerations for deploying ...,"Choose a reliable hosting platform, implement ..."


In [3]:
import csv
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
columns_to_embed = ["question"]
columns_to_metadata = ["combined_response"]

In [5]:
# Process the CSV into the embedable content vs the metadata and put it into Document format so that we can chunk it into pieces.
docs = []
with open('/content/intern_data_final - intern_data.csv', newline="", encoding='utf-8-sig') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for i, row in enumerate(csv_reader):
        to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
        values_to_embed = {k: (row[k]) for k in columns_to_embed if k in row}
        to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
        newDoc = Document(page_content=to_embed, metadata=to_metadata)
        docs.append(newDoc)

In [6]:
print(docs)

[Document(page_content='question: How can I implement user authentication in a MERN stack application?', metadata={'combined_response': 'Consider using industry-standard libraries like Passport.js and secure password hashing techniques like bcrypt. Implement proper token-based authentication and authorization mechanisms.'}), Document(page_content='question: How to best structure a large React application for maintainability and performance?', metadata={'combined_response': 'Utilize modular design principles, break down components into smaller, reusable units, and leverage lazy loading for efficient code execution.'}), Document(page_content='question: What are some effective approaches to optimize data fetching and state management in MERN applications?', metadata={'combined_response': "Explore state management libraries like Redux or Zustand for complex applications, or use React's context API for simpler scenarios. Implement efficient data fetching strategies like pagination and cachi

In [7]:
for document in docs:
    content = document.page_content
    metadata = document.metadata
    print(content, metadata)

question: How can I implement user authentication in a MERN stack application? {'combined_response': 'Consider using industry-standard libraries like Passport.js and secure password hashing techniques like bcrypt. Implement proper token-based authentication and authorization mechanisms.'}
question: How to best structure a large React application for maintainability and performance? {'combined_response': 'Utilize modular design principles, break down components into smaller, reusable units, and leverage lazy loading for efficient code execution.'}
question: What are some effective approaches to optimize data fetching and state management in MERN applications? {'combined_response': "Explore state management libraries like Redux or Zustand for complex applications, or use React's context API for simpler scenarios. Implement efficient data fetching strategies like pagination and caching."}
question: How can I create accessible and inclusive user interfaces in MERN applications? {'combined_

In [8]:
splitter = CharacterTextSplitter(separator = "\n",
                                chunk_size=500,
                                chunk_overlap=0,
                                length_function=len)
documents = splitter.split_documents(docs)

In [9]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
db = FAISS.from_documents(docs, hf)

In [11]:
print(type(db))

<class 'langchain_community.vectorstores.faiss.FAISS'>


In [12]:
print(db)

In [13]:
query = "For a particular task how to measure LLM performance"
embedding_vector = hf.embed_query(query)

In [14]:
d=45
k=1

In [15]:
import faiss

index = faiss.IndexFlatIP(d)
index.add(db)

AttributeError: 'FAISS' object has no attribute 'shape'